# Variational Bayesian Last Layer (VBLL) Regression

In [ ]:
%pip install lightning-uq-box

In [ ]:
import torch
import torch.nn as nn
import os
from functools import partial
from torch.optim import Adam
from lightning_uq_box.uq_methods import VBLLRegression
from lightning_uq_box.datamodules import ToyDUE
from lightning import Trainer
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import CSVLogger

from lightning_uq_box.models import MLP
from lightning_uq_box.viz_utils import (
    plot_toy_regression_data,
    plot_predictions_regression,
    plot_training_metrics,
)

import tempfile

%load_ext autoreload
%autoreload 2

In [ ]:
# temporary directory for saving
my_temp_dir = tempfile.mkdtemp()

seed_everything(42)

## Datamodule

In [ ]:
datamodule = ToyDUE(batch_size=32, n_samples=128, normalize=True)
datamodule.__dict__.keys()
X_train, y_train, X_test, y_test, train_loader, test_loader = (
    datamodule.X_train,
    datamodule.y_train,
    datamodule.X_test,
    datamodule.y_test,
    datamodule.train_dataloader(),
    datamodule.val_dataloader(),
)

In [ ]:
fig = plot_toy_regression_data(X_train, y_train, X_test, y_test)

## Model

In [ ]:
network = MLP(n_inputs=1, n_hidden=[64, 64], n_outputs=64, activation_fn=nn.ReLU())
network

In [ ]:
vbll_model = VBLLRegression(
    model=network,
    regularization_weight=(1 / X_train.shape[0]) * 10,
    optimizer=partial(torch.optim.AdamW, lr=1e-3, weight_decay=1e-4),
    prior_scale=1.0,
    wishart_scale=0.1
)

## Trainer

In [ ]:
logger = CSVLogger(my_temp_dir)
trainer = Trainer(
    max_epochs=1000,  # number of epochs we want to train
    logger=logger,  # log training metrics for later evaluation
    log_every_n_steps=1,
    enable_checkpointing=False,
    enable_progress_bar=True,
    limit_val_batches=0.0,  # no validation runs
    default_root_dir=my_temp_dir,
    gradient_clip_val=1.0
)

In [ ]:
trainer.fit(vbll_model, datamodule)

In [ ]:
fig = plot_training_metrics(
    os.path.join(my_temp_dir, "lightning_logs"), ["train_loss", "trainRMSE"]
)

## Prediction

In [ ]:
preds = vbll_model.predict_step(X_test)

## Evaluate Predictions

In [ ]:
fig = plot_predictions_regression(
    X_train,
    y_train,
    X_test,
    y_test,
    preds["pred"],
    preds["pred_uct"].squeeze(-1),
    epistemic=preds["pred_uct"].squeeze(-1),
    show_bands=True,
    title="VBLL",
)